In [ ]:
from elasticsearch import Elasticsearch
from bs4 import BeautifulSoup
from datetime import datetime
from flask import Flask, render_template, request
import os


In [ ]:

es = Elasticsearch([{'host': 'localhost', 'port':9200, 'scheme': 'http'}], http_auth=("vijay", "password"))
es.info()

In [ ]:
directory_path = "Assignment 3 dataset/AP_DATA/ap89_collection"

documents = list()

file_list = os.listdir(directory_path)

for file_name in file_list:

    file_path = os.path.join(directory_path, file_name)

    with open(file_path, "r") as file:

        tag_dict = dict()
        
        soup = BeautifulSoup(file, "xml")

        unique_tag_names = set(tag.name for tag in soup.find_all())

        for tag_name in unique_tag_names:

            content_list = soup.find_all(tag_name)

            for element in content_list:

                if tag_name in tag_dict:
                    
                    tag_dict[tag_name].append(element.text)

                else:

                    tag_dict[tag_name] = [element.text]
        
        
        documents.append(tag_dict)


print(documents[:10])    


In [ ]:
def index_documents(index, id, document):
    
    resp = es.index(index=index, id=id, document=document)
    print(resp['result'])


In [ ]:
for i in range(len(documents)):
    
    es.index(index = "my_index", id = i + 1, document = documents[i])


In [ ]:
resp = es.get(index="my_index", id=1)
print(resp['_source'])

In [ ]:
def search_query(q):
    
    body = {
        "size": 20,
        "query": {
            "dis_max": {
                "queries": [
                    {"match": {"head": q}},
                    {"match": {"text": q}}
                ],
                "tie_breaker": 0.3
            }
        }
    }

    resp = es.search(index="my_index", body=body)

    results = []

    for hit in resp['hits']['hits']:
        
        title = (hit["_source"].get('HEAD') or ['No title available'])[0]
        author = (hit["_source"].get('BYLINE') or ['No author available'])[0]
        date = (hit["_source"].get('FILEID') or ['No date available'])[0]
        text = (hit["_source"].get('TEXT') or ['No text available'])[0]

        text.replace('\n', '')

        formatted_result = f"Title: {title}\nAuthor: {author}\nDate: {date}\n\n{text}\n\n"

        formatted_result = formatted_result.replace('\n', '<br>')

        results.append(formatted_result)

    return results


In [ ]:
search_query("cat")

In [ ]:
app = Flask(__name__)


@app.route('/', methods=['GET', 'POST'])
def homepage():

    results = []

    if request.method == 'POST':

        query = request.form.get('search_query')
        
        results = search(query)
    
    return render_template('test.html', results=results)


def search(query):

    return search_query(query)


app.run(debug=False)